## dataset at https://www.kaggle.com/datasets/varun80042/food-cuisine

In [14]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from PIL import Image

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
data_dir = '/kaggle/input/food-cuisine'
batch_size = 32
num_classes = 6 
epochs = 10
learning_rate = 0.001
weight_decay = 1e-4  
model_save_path = 'model.pth' 

In [25]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [26]:
dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

In [27]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [28]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [29]:
val_dataset.dataset.transform = data_transforms['val_test']
test_dataset.dataset.transform = data_transforms['val_test']

In [30]:
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
}

In [31]:
class_names = dataset.classes
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}

In [32]:
dataset_sizes

{'train': 9645, 'val': 2066, 'test': 2068}

In [33]:
model = models.efficientnet_b4(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 182MB/s] 


In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [41]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=epochs):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    
    return model

In [42]:
model = train_model(model, criterion, optimizer, scheduler)

Epoch 0/9
----------
train Loss: 0.5865 Acc: 0.7933
val Loss: 0.7729 Acc: 0.7469

Epoch 1/9
----------
train Loss: 0.3532 Acc: 0.8767
val Loss: 0.7559 Acc: 0.7735

Epoch 2/9
----------
train Loss: 0.2760 Acc: 0.9082
val Loss: 0.8776 Acc: 0.7638

Epoch 3/9
----------
train Loss: 0.2641 Acc: 0.9114
val Loss: 0.8776 Acc: 0.7609

Epoch 4/9
----------
train Loss: 0.1082 Acc: 0.9666
val Loss: 0.7754 Acc: 0.8078

Epoch 5/9
----------
train Loss: 0.0473 Acc: 0.9857
val Loss: 0.8365 Acc: 0.8069

Epoch 6/9
----------
train Loss: 0.0282 Acc: 0.9916
val Loss: 0.8838 Acc: 0.8136

Epoch 7/9
----------
train Loss: 0.0231 Acc: 0.9926
val Loss: 0.9004 Acc: 0.8161

Epoch 8/9
----------
train Loss: 0.0159 Acc: 0.9957
val Loss: 0.9937 Acc: 0.8166

Epoch 9/9
----------
train Loss: 0.0158 Acc: 0.9947
val Loss: 0.9767 Acc: 0.8209

Best val Acc: 0.8209
Model saved to model.pth


In [43]:
def calculate_test_accuracy(model):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / dataset_sizes['test']
    print(f'Test Accuracy: {test_acc:.4f}')
    return test_acc

In [44]:
calculate_test_accuracy(model)

Test Accuracy: 0.8129


tensor(0.8129, device='cuda:0', dtype=torch.float64)

# predictive system

In [53]:
def predict_cuisine(image_path):
    model = models.efficientnet_b4(pretrained=False)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    image = Image.open(image_path)
    image = data_transforms['val_test'](image).unsqueeze(0)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    predicted_cuisine = class_names[preds.item()]
    return predicted_cuisine

In [54]:
sample_image_path = '/kaggle/input/food-cuisine/Indian/Indian_1.jpg' 
print(predict_cuisine(sample_image_path))

/tmp/ipykernel_37/2569168485.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path, map_location=device))


Indian


In [58]:
import glob
import random

cuisines = os.listdir(data_dir)
num_samples = 5

for cuisine in cuisines:
    cuisine_folder = os.path.join(data_dir, cuisine)
        
    image_files = glob.glob(os.path.join(cuisine_folder, "*.jpg"))
        
    sampled_images = random.sample(image_files, min(num_samples, len(image_files)))
        
    for image_path in sampled_images:
        predicted_cuisine = predict_cuisine(image_path)  
        print(f"Image: {image_path}")
        print(f"Actual Cuisine: {cuisine}")
        print(f"Predicted Cuisine: {predicted_cuisine}")
        print('-' * 40)


/tmp/ipykernel_37/2569168485.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path, map_location=device))


Image: /kaggle/input/food-cuisine/American/American_62.jpg
Actual Cuisine: American
Predicted Cuisine: American
----------------------------------------
Image: /kaggle/input/food-cuisine/American/American_1188.jpg
Actual Cuisine: American
Predicted Cuisine: American
----------------------------------------
Image: /kaggle/input/food-cuisine/American/American_1072.jpg
Actual Cuisine: American
Predicted Cuisine: Chinese
----------------------------------------
Image: /kaggle/input/food-cuisine/American/American_266.jpg
Actual Cuisine: American
Predicted Cuisine: American
----------------------------------------
Image: /kaggle/input/food-cuisine/American/American_1289.jpg
Actual Cuisine: American
Predicted Cuisine: American
----------------------------------------
Image: /kaggle/input/food-cuisine/Japanese/Japanese_1559.jpg
Actual Cuisine: Japanese
Predicted Cuisine: Japanese
----------------------------------------
Image: /kaggle/input/food-cuisine/Japanese/Japanese_145.jpg
Actual Cuisine